In [1]:
import json
data_dir = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_clipped/"+'melody.json'
queries = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_queries/"+'melody.json'


In [2]:
with open(data_dir, 'r') as f:
    fname_to_notes = json.load(f)


In [3]:
midiEt_to_note = {
    12: "C",
    13: "C#",
    14: "D",
    15: "D#",
    16: "E",
    17: "F",
    18: "F#",
    19: "G",
    20: "G#",
    21: "A",
    22: "A#",
    23: "B"
}


# a function which converts a sequence of midiEt to a sequence of notes
def midiEt_to_note_sequence(midiEt_sequence):
    note_sequence = []
    for midiEt in midiEt_sequence:
        num = midiEt % 12
        note_sequence.append(midiEt_to_note[num + 12])
    return "".join(note_sequence)


In [4]:
from mongeau_sankoff import similarity, Note
import miditoolkit as mtk
# queries=[Note("G",2,rest=False)]

# t=total_time/(4*tempo)
# duration=int(end-start/t)

In [18]:
#obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid")
obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_melody.mid")


In [19]:
notes = obj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

ans = 99999
prev_end = 0
for n in notes:
    if n.start > prev_end:
        dur = n.start - prev_end
        ans = min(ans, dur)
    ans = min(ans, n.end - n.start)
    prev_end = n.end
t= ans
        


In [20]:
rep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        rep.append(Note("C",int((n.start-prev_end)/t),rest=True))
    rep.append(Note(midiEt_to_note[n.pitch%12 + 12],int((n.end-n.start)/t),rest=False))
    prev_end = n.end

In [21]:
#qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_query_melody.mid")
qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_query_melody.mid")


In [22]:
notes = qobj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

ans = 99999
prev_end = 0
for n in notes:
    if n.start > prev_end:
        dur = n.start - prev_end
        ans = min(ans, dur)
    ans = min(ans, n.end - n.start)
    prev_end = n.end
t = ans


In [23]:
qrep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        qrep.append(Note("C", int((n.start-prev_end)/t), rest=True))
    qrep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False))
    prev_end = n.end


In [24]:
similarity(qrep, rep)

42it [00:00, 83.17it/s]


-2.4723333333333337

In [12]:
from tqdm import tqdm
import os
import music21
fnames = os.listdir("C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/")


def getKey(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.tonic.name)
def getMode(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.mode)


In [13]:
def compute_similarities(qrep):
    fname_to_similarity = {}
    for fname in fnames:
        sim = 0
        obj = mtk.MidiFile(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        notes = obj.instruments[0].notes 
        ans = 99999
        prev_end = 0
        
        for n in notes:
            if n.start > prev_end:
                dur = n.start - prev_end
                ans = min(ans, dur)
            ans = min(ans, n.end - n.start)
            prev_end = n.end
        t = ans
        mode = getMode(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        
        rep = []
        prev_end = 0
        for n in notes:
            if n.start > prev_end:
                rep.append(Note("C", int((n.start-prev_end)/t), rest=True,scale=mode))
            rep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False,scale=mode))
            prev_end = n.end
        sim = similarity(qrep, rep)
        fname_to_similarity[fname] = sim
    return fname_to_similarity


In [14]:
fname_to_similarity = compute_similarities(qrep)

61it [00:06,  9.82it/s]
13it [00:01,  8.95it/s]
7it [00:00,  7.88it/s]
34it [00:03,  9.21it/s]
20it [00:02,  9.30it/s]

KeyboardInterrupt: 

In [ ]:
dict(sorted(fname_to_similarity.items(), key=lambda item: item[1], reverse=True))

{'MIDI-Unprocessed_Recital9-11_MID--AUDIO_10_R1_2018_wav--3_melody.mid': 0.8618461538461538,
 'MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid': 0.7832307692307692,
 'MIDI-Unprocessed_Schubert1-3_MID--AUDIO_07_R2_2018_wav_melody.mid': 0.7813846153846155,
 'MIDI-Unprocessed_Recital17-19_MID--AUDIO_18_R1_2018_wav--3_melody.mid': 0.7796923076923077,
 'MIDI-Unprocessed_Recital1-3_MID--AUDIO_02_R1_2018_wav--1_melody.mid': 0.7581538461538462,
 'MIDI-Unprocessed_Recital1-3_MID--AUDIO_03_R1_2018_wav--4_melody.mid': 0.7412307692307691,
 'MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--2_melody.mid': 0.7410769230769232,
 'MIDI-Unprocessed_Recital12_MID--AUDIO_12_R1_2018_wav--1_melody.mid': 0.7256923076923077,
 'MIDI-Unprocessed_Recital8_MID--AUDIO_08_R1_2018_wav--4_melody.mid': 0.7222307692307692,
 'MIDI-Unprocessed_Recital17-19_MID--AUDIO_19_R1_2018_wav--2_melody.mid': 0.6912307692307693,
 'MIDI-Unprocessed_Recital9-11_MID--AUDIO_10_R1_2018_wav--2_melody.mid': 0.691230769